# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [12]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [13]:

if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.12.1
  %pip install torch==2.0.1
  %pip install tensorflow-gpu==2.5.3
  %pip install agents==1.4.0

Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/Kojoley/atari-py.git to c:\users\jgilg\appdata\local\temp\pip-req-build-_014gaug
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git 'C:\Users\jgilg\AppData\Local\Temp\pip-req-build-_014gaug'


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     ---------------------------------------- 1.3/1.3 MB 16.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-1.24.3-cp38-cp38-win_amd64.w

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'c:\\Users\\jgilg\\anaconda3\\envs\\miar_rl\\Lib\\site-packages\\tensorflow\\python\\lib\\core\\_pywrap_bfloat16.pyd'
Consider using the `--user` option or check the permissions.



Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl.metadata (2.0 kB)
  Using cached absl_py-0.15.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl.metadata (872 bytes)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached grpcio-1.34.1-cp38-cp38-win_amd64.whl.metadata (4.0 kB)
INFO: pip is looking at multiple versions of tensorboard to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorboard-2.14.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.12.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.12.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.12.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.11.2-py3-none-any.whl.metadata (1.9 kB)
  Using cac

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'c:\\Users\\jgilg\\anaconda3\\envs\\miar_rl\\Lib\\site-packages\\tensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



Note: you may need to restart the kernel to use updated packages.


---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [25]:
from __future__ import division

from PIL import Image
import numpy as np
import gym
import csv
import os

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint, Callback

#### Configuración base

In [26]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
print("Numero de acciones disponibles: " + str(nb_actions))

Numero de acciones disponibles: 6


In [27]:
print("Formato de las observaciones:")
env.observation_space

Formato de las observaciones:


Box(0, 255, (210, 160, 3), uint8)

In [28]:
# Preparamos la gpu local del ordenador 

from tensorflow.python.client import device_lib
from tensorflow import config
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.5.3
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [29]:
from rl.core import Processor

class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

0. Indicamos la versión del modelo

In [30]:
# Borrar posibles pesos que quueden de una version anterior

K.clear_session()

#------------------------------------------------------

# Versión entrenamiento

version = "5"

# Crear carpeta con la version

folder_path = f"version_{version}"
os.makedirs(folder_path, exist_ok=True)


1. Implementación de la red neuronal

In [31]:
# Red neuronal DQN (+ enables double y dueling)

def model_RRNN_DQN(input_shape = (4,84,84), nb_actions = nb_actions):
    inputs = Input(shape=input_shape)
    x = Permute((2,3,1))(inputs)

    # Red convolucional
    x = Convolution2D(32, (8,8), strides = (4,4), activation = "relu")(x)
    x = Convolution2D(64, (4,4), strides = (2,2), activation = "relu")(x)
    x = Convolution2D(64, (3,3), strides = (1,1), activation = "relu")(x)
    x = Flatten()(x)
    x = Dense(512, activation = "relu")(x)
    x = Dense(nb_actions, activation = "linear")(x)

    model_DQN = Model(inputs = inputs, outputs = x)
    return model_DQN

model_DQN = model_RRNN_DQN(input_shape = (4,84,84), nb_actions = nb_actions)
print(model_DQN.summary())

#-----------------------------------------------------------------------------------------------

# Q-value_layer para dueling network DQN
def q_value_layer(inputs_q):
    v, a = inputs_q
    return v + (a - tf.reduce_mean(a, axis=1, keepdims=True)) # Uso del average (no max ni naive)


# Red neuronal dueling_DQN
def dueling_network_DQN(input_shape = (4,84,84), nb_actions = nb_actions):
    inputs = Input(shape=input_shape)
    x = Permute((2,3,1))(inputs)

    # Red convolucional
    x = Convolution2D(32, (8,8), strides = (4,4), activation = "relu")(x)
    x = Convolution2D(64, (4,4), strides = (2,2), activation = "relu")(x)
    x = Convolution2D(64, (3,3), strides = (1,1), activation = "relu")(x)
    x = Flatten()(x)

    # Value
    v = Dense(512, activation = "relu")(x)
    v = Dense(1, activation = "linear")(v)

    # Advantage
    a = Dense(512, activation = "relu")(x)
    a = Dense(nb_actions, activation = "linear")(a)

   

    q = q_value_layer([v, a])

    model_dueling_DQN = Model(inputs = inputs, outputs = q)
    return model_dueling_DQN

model_dueling_network_DQN = dueling_network_DQN(input_shape = (4,84,84), nb_actions = nb_actions)
print(model_dueling_network_DQN.summary())



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4, 84, 84)]       0         
_________________________________________________________________
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               160614

2. Implementación de la solución DQN

In [ ]:
processor = AtariProcessor()
memory = SequentialMemory(limit=800000, window_length = 4)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr = "eps", value_max = 1.0, value_min = 0.1, value_test = 0.05, nb_steps = 2250000)  # Value test se añade un 0.05; 75% del entrenamiento, LinearAnnealedPolicy

DQN = DQNAgent(model = model_DQN,
               nb_actions = nb_actions,
               memory = memory,
               gamma = 0.99,
               batch_size = 32,   
               train_interval = 4,
               enable_double_dqn = False,       # Desabilitamos el double dqn
               enable_dueling_network = True,   # Sigue siendo dueling, pero haciendolo automaticamente la libería 
               dueling_type = "avg",
               nb_steps_warmup = 50000,
               target_model_update = 5000,  # Se baja a 5000 
               policy = policy,
               processor = processor)

DQN.compile(Adam(learning_rate = 1e-4),metrics=['mae']) 

In [33]:
# Callback para guardar las métricas obtenidas durante el entrenamiento

class MetricLogger(Callback):
    def __init__(self, version):
        self.version = version
        self.folder = f"version_{version}"
        os.makedirs(self.folder, exist_ok=True)
        self.filename = os.path.join(self.folder, f"dqn_v_{version}_training_log.csv")
        self.fields = ['episode', 'steps', 'reward']

        with open(self.filename, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=self.fields)
            writer.writeheader()

    def on_episode_end(self, episode, logs):
        row = {
            'episode': episode,
            'steps': logs.get('nb_steps'),
            'reward': logs.get('episode_reward'),
           
        }

        with open(self.filename, 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=self.fields)
            writer.writerow(row)
            
metricas = MetricLogger(version)

In [34]:
# Callback para guardar los pesos del modelo cada cierto numero de episodios

class SaveWeightsInterval(Callback):
    def __init__(self, agent, version, interval):
        self.agent = agent
        self.version = version
        self.interval = interval
        self.folder = f"version_{version}"
        os.makedirs(self.folder, exist_ok=True)

    def on_episode_end(self, episode, logs):
        if episode and episode % self.interval == 0:   
            filename = os.path.join(self.folder, f"dqn_v_{version}_weights_{episode}_episodes.h5f")
            self.agent.save_weights(filename, overwrite=True)

guardar_pesos_iteraciones = SaveWeightsInterval(DQN, version, 120) # Se guardan los pesos cada 100 episodios

In [35]:
# Callback para guardar los pesos del episodio que ha dado mejor recompensa

class SaveBestWeights(Callback):
    def __init__(self, agent, version):
        self.agent = agent
        self.version = version
        self.folder = f"version_{version}"
        os.makedirs(self.folder, exist_ok=True)
        self.best_reward = -np.inf  
        self.best_weights_file = os.path.join(self.folder, f"dqn_v_{version}_best_weights.h5f")

    def on_episode_end(self, episode, logs):
        reward = logs.get('episode_reward')
        if reward is not None and reward > self.best_reward:
            self.best_reward = reward
            self.agent.save_weights(self.best_weights_file, overwrite=True)

guardar_mejores_pesos = SaveBestWeights(DQN, version)

In [36]:
folder = f"version_{version}"
weights_filename = os.path.join(folder, f'dqn_v_{version}_final_weights.h5f')

DQN.fit(env, nb_steps=3000000, visualize=False, verbose=2, callbacks=[metricas, guardar_pesos_iteraciones, guardar_mejores_pesos]) # Se aumenta entrenamiento a 2 millones de steps
DQN.save_weights(weights_filename, overwrite=True)

Training for 3000000 steps ...


c:\Users\jgilg\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


     420/3000000: episode: 1, duration: 1.812s, episode steps: 420, steps per second: 232, episode reward:  6.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.479 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1131/3000000: episode: 2, duration: 2.422s, episode steps: 711, steps per second: 294, episode reward:  7.000, mean reward:  0.010 [ 0.000,  1.000], mean action: 2.414 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1941/3000000: episode: 3, duration: 3.736s, episode steps: 810, steps per second: 217, episode reward: 10.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.410 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    2809/3000000: episode: 4, duration: 4.094s, episode steps: 868, steps per second: 212, episode reward: 11.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.381 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    3730/3000000: episode: 5, duration: 4.215s, episode steps: 921, 

c:\Users\jgilg\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   50314/3000000: episode: 72, duration: 6.416s, episode steps: 721, steps per second: 112, episode reward: 11.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.592 [0.000, 5.000],  loss: 0.006330, mae: 0.028790, mean_q: 0.047113, mean_eps: 0.979937
   51406/3000000: episode: 73, duration: 17.163s, episode steps: 1092, steps per second:  64, episode reward: 21.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 2.477 [0.000, 5.000],  loss: 0.006727, mae: 0.030753, mean_q: 0.043840, mean_eps: 0.979656
   51920/3000000: episode: 74, duration: 8.026s, episode steps: 514, steps per second:  64, episode reward:  7.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.578 [0.000, 5.000],  loss: 0.006294, mae: 0.029701, mean_q: 0.038278, mean_eps: 0.979335
   52734/3000000: episode: 75, duration: 13.352s, episode steps: 814, steps per second:  61, episode reward: 16.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 2.533 [0.000, 5.000],  loss: 0.007694, mae: 0.032840, me

In [44]:
folder = f"version_{version}"
weights_filename = os.path.join(folder, f'dqn_v_{version}_final_weights.h5f')

DQN.load_weights(weights_filename)
history = DQN.test(env, nb_episodes=10, visualize=False)

# Obtener media y desviación estandar de la recompensa de los episodios
recompensas = history.history.get('episode_reward', [])
print(f"Media de la recompensa obtenida en 10 episodios: {np.mean(np.array(recompensas))}")
print(f"Desviación estandar de la recompensa obtenida en 10 episodios: {np.std(np.array(recompensas))}")

Testing for 10 episodes ...
Episode 1: reward: 35.000, steps: 1427
Episode 2: reward: 26.000, steps: 1443
Episode 3: reward: 21.000, steps: 972
Episode 4: reward: 24.000, steps: 897
Episode 5: reward: 19.000, steps: 982
Episode 6: reward: 30.000, steps: 1225
Episode 7: reward: 10.000, steps: 489
Episode 8: reward: 25.000, steps: 1050
Episode 9: reward: 25.000, steps: 952
Episode 10: reward: 34.000, steps: 1689
Media de la recompensa obtenida en 10 episodios: 24.9
Desviación estandar de la recompensa obtenida en 10 episodios: 6.963476143421473


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---